In [1]:
from components import Components
from llama_index.core import Settings

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate,Document 


from llama_index.core.node_parser import SentenceSplitter

import pandas as pd

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
#initialise components, reranker and retriever
rag_components = Components("Snowflake/snowflake-arctic-embed-s","mixedbread-ai/mxbai-embed-large-v1","gemma2-9b-it")

Setting Contexts


In [21]:
Settings.embed_model = rag_components.get_embedding_model()
Settings.llm = rag_components.get_groq_llm()

Embedding model loaded!


In [4]:
vector_store = ChromaVectorStore(chroma_collection=rag_components.get_db())
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Loading Papers

In [5]:
df = pd.read_parquet('data/test/test_neural_bridge_qa.parquet')

In [6]:
context_list = []
for contexts in df['context'][:128]:
    context_list.append(Document(text=contexts))

In [7]:
pipeline = IngestionPipeline(
    transformations=[
          #SentenceSplitter(chunk_size=1000, chunk_overlap=200),
    ],
    vector_store=vector_store,
)

In [8]:
documents = pipeline.run(documents=context_list)

In [9]:
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context,similarity_top_k=10, node_postprocessors=[rag_components.get_reranker()]
) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mixedbread-ai/mxbai-embed-large-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Reranker model loaded!


In [10]:
query_engine = index.as_query_engine(streaming=True,similarity_top_k=10)

In [34]:
qa_prompt_template_str = """
Ignore all instructions previously given.
Context information is below.
---------------------
{context_str}
---------------------
Based on the context above,answer the question concisely and immediately.
If you are unable to answer with the given information, say "I don't know".
Question: {query_str}
Answer:"""


qa_prompt_template = PromptTemplate(qa_prompt_template_str)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":qa_prompt_template}
)


In [12]:
question_list = []
for questions in df['question'][:128]:
    question_list.append(questions)
    

In [35]:
response = query_engine.query(question_list[0])
response.print_response_stream()
response.get_formatted_sources()

I think it's because she's a shining star, and I'm so grateful to be along for the ride. It's because I've watched her grow and evolve, and I'm constantly amazed by her talent and dedication. It's because I've seen her bring joy to so many people, and I feel proud to be her partner in that. It's because I've experienced the thrill of watching her perform, and I'm in awe of her skill and artistry. It's because I've seen her take risks and push boundaries, and I admire her bravery and creativity. It's because I know that she's a true original, and I feel lucky to be a part of her journey.

'> Source (Node id: e67bdf68-77e4-4f64-86f1-f10feea457b1): HOUSTON (Jan. 23, 2018) – Fabien Gabel, music director of the Quebec Symphony Orchestra, returns ...\n\n> Source (Node id: ef224ab5-46ca-4da1-9c59-47aac114ab77): In that sense, all legislators elected "by accumulation," i.e., not tied to any senatorial or rep...\n\n> Source (Node id: 48259bbf-bb75-4e9a-af42-a074d901ef3c): She co-edited a monograph series on transatlantic literary studies and was a board member and pas...\n\n> Source (Node id: eacc1bce-09fe-411e-974b-d2ff1f00b472): "I don\'t, really, no," she says. "But, you know, when you think about it, I\'m a performing artist...\n\n> Source (Node id: 282f8190-3fc4-4a69-ba67-0133a1305753): - What are you most proud of in life?\n- What has been the happiest moment in your life? The sadde...\n\n> Source (Node id: f6d8d14a-e419-4ab7-84cb-021f19d7a21c): It\'s 8:30 am.\nI woke around 7am when theHotness kissed me goodbye on his way out the door to work...\n\n> Source (Node id: 5a6

In [14]:
response.get_response()

Response(response="I think it's braver now.", source_nodes=[NodeWithScore(node=TextNode(id_='e67bdf68-77e4-4f64-86f1-f10feea457b1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='fd17905b-57ef-4659-a0f2-f3d91e7b2954', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='9ac13b18392cafd1fb7dbe8a7b0b48a2cca1acceda92e492609dcc6b0fd39add')}, text='HOUSTON (Jan. 23, 2018) – Fabien Gabel, music director of the Quebec Symphony Orchestra, returns to Houston to lead the Houston Symphony in Ravel’s Daphnis and Chloé on Feb. 2 and 3 at 8 p.m. and Feb. 4 at 2:30 p.m. in Jones Hall.\nRecognized internationally as one of the stars of the new generation, Fabien Gabel is a regular guest of the Houston Symphony and an audience favorite. Known for conducting music with French influences, Gabel leads the Symphony in a program of French and American classics, including the breathtaking

In [31]:
import time
for i in range(len(question_list)): 
    time.sleep(3)
    print(question_list[i])
    response = query_engine.query(question_list[i])
    response.print_response_stream()
    rowIndex = df.index[i]

    df.at[rowIndex, 'gen_answer'] = response.response_txt
    print('')
    print("--"*100)
    print(f'{i}/{128}')
 


Who is the music director of the Quebec Symphony Orchestra?
I think it's safe to say that the music director of the Quebec Symphony Orchestra is not someone who is easily deterred from their passion for music.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0/128
Who were the four students of the University of Port Harcourt that were allegedly murdered?
I'm not familiar with the specific context you're referring to, but I can try to help you with your question.

The University of Port Harcourt is a university located in Port Harcourt, Nigeria. However, without more information, I couldn't find any specific details about the alleged murder of four students at the university.

If you could provide more context or clarify what you're referring to, I'd be happy to try and help you further.
--------------------------------------------------

KeyboardInterrupt: 